In [5]:
Item.objects.filter(pk=401).update(list_thumbnail='items/thumnail_401.jpg')

1

In [33]:
suffix = 1085
for pk in range(1085, 1092):
    Item.objects.filter(pk=pk).update(list_thumbnail='items/thumnail_'+str(suffix)+'.jpg')
    suffix += 1
    print(pk)

1085
1086
1087
1088
1089
1090
1091


In [20]:
Item.objects.filter(pk=924).update(list_thumbnail='')

1

In [19]:
Item.objects.filter(pk=839).update(list_thumbnail='items/thumnail_'+str(841)+'.jpg')

1

In [23]:
Item.objects.filter(pk=924).update(list_thumbnail='')

1

In [24]:
Item.objects.filter(pk=925).update(list_thumbnail='')

1

In [25]:
Item.objects.filter(pk=926).update(list_thumbnail='')

1

In [26]:
Item.objects.filter(pk=927).update(list_thumbnail='')

1

In [29]:
Item.objects.filter(pk=976).update(list_thumbnail='')

1

In [8]:
col_detail_url = [
    'https://www.baeminchan.com/mealplan/detail.php?hash=G0F42&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=F1256&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=F23AA&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=F62EE&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=FF7CB&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=G27B7&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=FDAEB&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=GAF8D&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=GA3FA&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=GCA07&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=F6B07&cno=46110000',    
]

In [9]:
from bs4 import BeautifulSoup
import requests, re

url = 'https://www.baeminchan.com/mealplan/list.php'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
soup_products = soup.find('ul', id='products')
col_products = []
col_products = soup_products.findAll('li')
col_ga_id = []
for col in col_products:
    col_ga_id.append(col.select('a')[0].get('ga_id'))
  

In [39]:
for col in col_ga_id:
    print(col)

10815
5825
5826
1890
5815
9915
5881
12481
12841
6934
5648


In [10]:
for url in col_detail_url:
    print(url)
    ga_id = col_ga_id[col_detail_url.index(url)]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    soup_detail = soup.find('div', id='app')
    
    added_words = ''
    try:
        added_words = soup_detail.find_all("p", "desc_bt_txt")[0].get_text()
    except:
        pass
            
    point = 0
    try:
        int(soup_detail.find_all("dl", "desc_info")[0].select('dd')[0].get_text().replace('원',''))
    except:
        pass
    else:
        point = int(soup_detail.find_all("dl", "desc_info")[0].select('dd')[0].get_text().replace('원',''))
            
    delivery_type = ''
    try:        
        delivery_type = soup_detail.find_all("dl", "desc_info")[0].select('dd')[1].get_text().replace('\r','').replace('\n','')
    except:
        pass                    
    
    receive_day = ''
    try:
        receive_day = soup_detail.find_all("dl", "desc_info")[0].select('dd')[2].get_text().replace('\r','').replace('\n','')
    except:
        pass
    
    regular_delivery = ''
    try:
        regular_delivery = soup_detail.find_all("dl", "desc_info")[0].select('dd')[3].get_text()
    except:
        pass
    
    if regular_delivery == '가능':
        regular_delivery = True
    else:
        regular_delivery = False
        
    soup_detail_description = soup.find('div', id='detail_official_information')
    col_is_equal = [
        '식품의 유형',
        '생산자 및 소재지(수입품의 경우 수입자를 함께 표기)',
        '제조연월일, 유통기한 또는 품질유지기한',
        '포장단위별 용량(중량), 수량',
        '원재료명 및 함량',
        '영양성분',
        '유전자재조합식품 여부',
        '영유아식 또는 체중조절식품 등에 해당하는 경우',
        '수입식품 문구',
        '소비자상담 관련 전화번호',    
        '알레르기 주의사항',
        '알레르기 유발물질',
        '품목신고번호',
    ]
    col_th = []
    col_td = []
    
    col_th = soup_detail_description.find_all('tbody')[0].select('th')
    col_td = soup_detail_description.find_all('tbody')[0].select('td')
    
    col_description_data = []

    for col in col_is_equal:
        descripiton = ''
        for col_th_num in col_th:
            if col == col_th_num.get_text():
                descripiton = col_td[col_th.index(col_th_num)].get_text().replace('\r','').replace('\n','')
                break

        col_description_data.append(descripiton)
    print(added_words)
    print(ga_id)
    Description.objects.create(
        item=Item.objects.get(ga_id=ga_id), 
        added_words=added_words,
        point=point,
        delivery_type=delivery_type,
        receive_day=receive_day,
        regular_delivery=regular_delivery,
        item_type=col_description_data[0],
        factory_address=col_description_data[1],
        dom=col_description_data[2],
        capacity=col_description_data[3],
        ingredient=col_description_data[4],
        caution=col_description_data[10],                
        allergy_material=col_description_data[11],        
    )
    
#     print(point)
#     print(delivery_type)
#     print(receive_day)
#     print(regular_delivery)
#     print(col_description_data[0])
#     print(col_description_data[1])
#     print(col_description_data[2])
#     print(col_description_data[3])
#     print(col_description_data[4])
#     print(col_description_data[10])
#     print(col_description_data[11])

#     try:
#         Description.objects.create(
#             item=Item.objects.get(ga_id=ga_id), 
#             added_words=added_words,
#             point=point,
#             delivery_type=delivery_type,
#             receive_day=receive_day,
#             regular_delivery=regular_delivery,
#             item_type=col_description_data[0],
#             factory_address=col_description_data[1],
#             dom=col_description_data[2],
#             capacity=col_description_data[3],
#             ingredient=col_description_data[4],
#             caution=col_description_data[10],                
#             allergy_material=col_description_data[11],        
#         )
#         print('success')
#     except:
#         print('fail')
#         pass   

https://www.baeminchan.com/mealplan/detail.php?hash=G0F42&cno=46110000
국내 최고 요리연구가 방영아의 제안
10815
https://www.baeminchan.com/mealplan/detail.php?hash=F1256&cno=46110000
식단 정기배송으로 반찬고민 끝
5825
https://www.baeminchan.com/mealplan/detail.php?hash=F23AA&cno=46110000
5만명 이상의 고객님이 인정한 진짜 집밥!
5826
https://www.baeminchan.com/mealplan/detail.php?hash=F62EE&cno=46110000
반찬 4가지,국·찌개류 1가지 정기배송
1890
https://www.baeminchan.com/mealplan/detail.php?hash=FF7CB&cno=46110000
5만 고객님이 인정한 더푸드 반찬배송
5815
https://www.baeminchan.com/mealplan/detail.php?hash=G27B7&cno=46110000
분당의 인기 반찬가게 리쿡54의 수제반찬
9915
https://www.baeminchan.com/mealplan/detail.php?hash=FDAEB&cno=46110000
식단 정기배송으로 반찬고민 끝
5881
https://www.baeminchan.com/mealplan/detail.php?hash=GAF8D&cno=46110000
국내 최고 요리연구가 방영아의 제안
12481
https://www.baeminchan.com/mealplan/detail.php?hash=GA3FA&cno=46110000
옥수동의 델리형 반찬가게
12841
https://www.baeminchan.com/mealplan/detail.php?hash=GCA07&cno=46110000
오늘은 뭐먹지 고민은 넣어두세요
6934
https://www.baeminchan.com/mealplan/detai

In [11]:
item = Item.objects.get(ga_id='10815')

In [12]:
item.id

1081

In [25]:
item = Description.objects.get(item=Item.objects.get(pk=1070))

In [26]:
item

<Description: [Young313]아이반찬(반찬4,국1가지)>

In [7]:
Description.objects.filter(item=Item.objects.get(pk=1085)).delete()

(1, {'items.Description': 1})

In [12]:
url = 'https://www.baeminchan.com/sidedish/detail.php?cno=46010100&hash=I95A7'
    
ga_id = 18075
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
soup_detail = soup.find('div', id='app')

added_words = ''
try:
    added_words = soup_detail.find_all("p", "desc_bt_txt")[0].get_text()
except:
    pass

point = 0
try:
    int(soup_detail.find_all("dl", "desc_info")[0].select('dd')[0].get_text().replace('원',''))
except:
    pass
else:
    point = int(soup_detail.find_all("dl", "desc_info")[0].select('dd')[0].get_text().replace('원',''))

delivery_type = ''
try:        
    delivery_type = soup_detail.find_all("dl", "desc_info")[0].select('dd')[1].get_text().replace('\r','').replace('\n','')
except:
    pass                    

receive_day = ''
try:
    receive_day = soup_detail.find_all("dl", "desc_info")[0].select('dd')[2].get_text().replace('\r','').replace('\n','')
except:
    pass

regular_delivery = ''
try:
    regular_delivery = soup_detail.find_all("dl", "desc_info")[0].select('dd')[3].get_text()
except:
    pass

if regular_delivery == '가능':
    regular_delivery = True
else:
    regular_delivery = False

soup_detail_description = soup.find('div', id='detail_official_information')
col_is_equal = [
    '식품의 유형',
    '생산자 및 소재지(수입품의 경우 수입자를 함께 표기)',
    '제조연월일, 유통기한 또는 품질유지기한',
    '포장단위별 용량(중량), 수량',
    '원재료명 및 함량',
    '영양성분',
    '유전자재조합식품 여부',
    '영유아식 또는 체중조절식품 등에 해당하는 경우',
    '수입식품 문구',
    '소비자상담 관련 전화번호',    
    '알레르기 주의사항',
    '알레르기 유발물질',
    '품목신고번호',
]
col_th = []
col_td = []

col_th = soup_detail_description.find_all('tbody')[0].select('th')
col_td = soup_detail_description.find_all('tbody')[0].select('td')

col_description_data = []

for col in col_is_equal:
    descripiton = ''
    for col_th_num in col_th:
        if col == col_th_num.get_text():
            descripiton = col_td[col_th.index(col_th_num)].get_text().replace('\r','').replace('\n','')
            break

    col_description_data.append(descripiton)
print(added_words)
print(ga_id)
Description.objects.create(
    item=Item.objects.get(ga_id=ga_id), 
    added_words=added_words,
    point=point,
    delivery_type=delivery_type,
    receive_day=receive_day,
    regular_delivery=regular_delivery,
    item_type=col_description_data[0],
    factory_address=col_description_data[1],
    dom=col_description_data[2],
    capacity=col_description_data[3],
    ingredient=col_description_data[4],
    caution=col_description_data[10],                
    allergy_material=col_description_data[11],        
)    

아삭- 소리와 함께 되찾은 입맛
18075


<Description: [집밥의완성]오이사과부추버무리 (1~2인분) 280g>

In [12]:
from bs4 import BeautifulSoup
import requests, re

col_ga_id = []
col_company = []
col_item_name = []
col_sales_price = []
col_origin_price = []
col_discounts = []
col_img_url = []
col_detail_url = []

col_page =[]

p_company = re.compile(r'\[(?P<company>.*)\](?P<item_name>.*)')
p_price = re.compile(r'\d.*\d')


cnt = 0


url = 'https://www.baeminchan.com/meat/list.php?cno=46100700'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
soup_products = soup.find('ul', id='products')
col_products = []
col_products = soup_products.findAll('li')
col_imgthumb = []
col_imgthumb = soup_products.find_all("div", "imgthumb")

for col in col_products:
    col_ga_id.append(col.select('a')[0].get('ga_id'))
    col_company.append(p_company.search(col.select('a')[0].get('ga_name')).group('company'))
    col_item_name.append(p_company.search(col.select('a')[0].get('ga_name')).group('item_name').lstrip())       

for col in col_products:        
    try:
        p_price.search(col.select('p')[0].get_text()).group()
    except:
        col_sales_price.append(0)
    else:
        col_sales_price.append(int(p_price.search(col.select('p')[0].get_text()).group().replace(',','')))           

for col in col_products:
    try:
        p_price.search(col.select('p')[1].get_text()).group()
    except IndexError:
        try:
            p_price.search(col.select('p')[0].get_text()).group().replace(',','')
        except:
            col_origin_price.append(0)
        else:
            col_origin_price.append(p_price.search(col.select('p')[0].get_text()).group().replace(',',''))    
    else:
        col_origin_price.append(p_price.search(col.select('p')[1].get_text()).group().replace(',',''))

for col in col_imgthumb:
    try:
        col.select('span')[0].get_text()
    except IndexError:
        col_discounts.append(float(0))
    else:
        col_discounts.append(float(col.select('span')[0].get_text()) * 0.01)

for col in col_imgthumb:
    col_img_url.append(col.select('img')[0].get('src'))
    col_detail_url.append(url.replace('list.php',col.select('a')[0].get('href')))            
            

In [14]:
# col_ga_id = []
# col_company = []
# col_item_name = []
# col_sales_price = []
# col_origin_price = []
# col_discounts = []
# col_img_url = []
# col_detail_url = []
for col in col_detail_url:
    print(col)

https://www.baeminchan.com/meat/detail.php?hash=HFAE4&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=IE3E6&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=I155E&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=I3E2A&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=ID6FE&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=IF045&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=I4A79&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=IDDA8&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=IEFE4&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=IF329&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=ICD4E&cno=46100700?cno=46100700


In [16]:
item_cnt = 0
thumb_nail = 1092

for col in col_item_name:        
    Item.objects.create(pk=thumb_nail, item_name=col_item_name[item_cnt], company=col_company[item_cnt], origin_price=col_origin_price[item_cnt-1], sale_price=col_sales_price[item_cnt-1], discount_rate=col_discounts[item_cnt-1], ga_id=col_ga_id[item_cnt],list_thumbnail='items/thumnail_'+str(thumb_nail)+'.jpg')
    item_cnt += 1
    thumb_nail += 1    
    

# col_page_category = []
# col_category_ga_id = []
# category_cnt = 1


# for col in col_category:
#     response = requests.get(col)
#     soup = BeautifulSoup(response.text, 'lxml')
    
#     max_page = 0
    
#     try:
#         soup.find('div', 'pagination').findAll('a')
#     except:
#         max_page = 0
#     else:
#         max_page = len(soup.find('div', 'pagination').findAll('a'))-1
    
#     col_url_page = []
    
#     if max_page == 0:
#         col_url_page.append(col)
#     else:            
#         for num in range(1, max_page+1):            
#             col_url_page.append(col+'&'+'page='+str(num))
            
#     for url in col_url_page:    
#         print(url)
#         response = requests.get(url)
#         soup = BeautifulSoup(response.text, 'lxml')
#         soup_products = soup.find('ul', id='products')
#         col_products = []
#         col_products = soup_products.findAll('li')       

#         for col in col_products:
#             ga_id = col.select('a')[0].get('ga_id')
#             try:
#                 Item.objects.get(ga_id=ga_id).categories.add(Category.objects.get(pk=category_cnt))
#             except:
#                 pass            
                 
    
#     print('----------------------------'+str(category_cnt)+'----------------------------------')
#     category_cnt += 1
    
    

In [17]:
for pk in range(1092, 1103):
    Item.objects.get(pk=pk).categories.add(Category.objects.get(pk=26))
    Item.objects.get(pk=pk).categories.add(Category.objects.get(pk=31))

In [19]:
import urllib.request

cnt = 1092
for col in col_img_url:
    urllib.request.urlretrieve(col, '../items/thumbnail'+str(cnt)+'.jpg')
    cnt += 1    

In [21]:
col_detail_url 
col_ga_id

for url in col_detail_url :    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    soup_detail = soup.find('div', id='app')

    added_words = ''
    try:
        added_words = soup_detail.find_all("p", "desc_bt_txt")[0].get_text()
    except:
        pass

    point = 0
    try:
        int(soup_detail.find_all("dl", "desc_info")[0].select('dd')[0].get_text().replace('원',''))
    except:
        pass
    else:
        point = int(soup_detail.find_all("dl", "desc_info")[0].select('dd')[0].get_text().replace('원',''))

    delivery_type = ''
    try:        
        delivery_type = soup_detail.find_all("dl", "desc_info")[0].select('dd')[1].get_text().replace('\r','').replace('\n','')
    except:
        pass                    

    receive_day = ''
    try:
        receive_day = soup_detail.find_all("dl", "desc_info")[0].select('dd')[2].get_text().replace('\r','').replace('\n','')
    except:
        pass

    regular_delivery = ''
    try:
        regular_delivery = soup_detail.find_all("dl", "desc_info")[0].select('dd')[3].get_text()
    except:
        pass

    if regular_delivery == '가능':
        regular_delivery = True
    else:
        regular_delivery = False

    soup_detail_description = soup.find('div', id='detail_official_information')
    col_is_equal = [
        '식품의 유형',
        '생산자 및 소재지(수입품의 경우 수입자를 함께 표기)',
        '제조연월일, 유통기한 또는 품질유지기한',
        '포장단위별 용량(중량), 수량',
        '원재료명 및 함량',
        '영양성분',
        '유전자재조합식품 여부',
        '영유아식 또는 체중조절식품 등에 해당하는 경우',
        '수입식품 문구',
        '소비자상담 관련 전화번호',    
        '알레르기 주의사항',
        '알레르기 유발물질',
        '품목신고번호',
    ]
    col_th = []
    col_td = []

    col_th = soup_detail_description.find_all('tbody')[0].select('th')
    col_td = soup_detail_description.find_all('tbody')[0].select('td')

    col_description_data = []

    for col in col_is_equal:
        descripiton = ''
        for col_th_num in col_th:
            if col == col_th_num.get_text():
                descripiton = col_td[col_th.index(col_th_num)].get_text().replace('\r','').replace('\n','')
                break

        col_description_data.append(descripiton)
    print(added_words)    
    Description.objects.create(
        item=Item.objects.get(ga_id=col_ga_id[col_detail_url.index(url)]), 
        added_words=added_words,
        point=point,
        delivery_type=delivery_type,
        receive_day=receive_day,
        regular_delivery=regular_delivery,
        item_type=col_description_data[0],
        factory_address=col_description_data[1],
        dom=col_description_data[2],
        capacity=col_description_data[3],
        ingredient=col_description_data[4],
        caution=col_description_data[10],                
        allergy_material=col_description_data[11],        
    )  

아무데나 쓱 넣어주면 향긋함이 두 배
국내산 버섯으로 고기구이를 더욱 풍성하게


고기랑 이 세트하나면 모든게 끝!

고기 먹을때 꼭 챙겨주세요
고기의 느끼한 맛을 잡아주는 청양고추
신선한 국내산 대파로 만든 파채
바로 활용 할 수 있는 손질된 단호박
 손질되어 바로 활용할 수 있어요


In [42]:
col_itemimage = []
pk = 14474
for url in col_detail_url :    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    soup_detail = soup.find('div', id='app')

    col_image_thumb = []
    col_image_thumb = soup_detail.find_all("div", "image_thumb")[0].select('img')
    for col in col_image_thumb:                
        col_itemimage.append(col.get('src'))
        order = col_image_thumb.index(col)        
        try:            
            ItemImage.objects.create(pk=pk, item=Item.objects.get(ga_id=col_ga_id[col_detail_url.index(url)]), photo_type='T', image_order=order, photo='items/detail_thumnail_'+str(col_ga_id[col_detail_url.index(url)])+'_'+str(order)+'.jpg')
        except:                   
            pass
                          
        pk += 1
        
    col_product_detail = []
    col_product_detail = soup_detail.find_all("div", "product_detail_img_box")[0].select('img')
    for col in col_product_detail:                
        col_itemimage.append(col.get('src'))
        order = col_product_detail.index(col)
        try:            
            ItemImage.objects.create(pk=pk, item=Item.objects.get(ga_id=col_ga_id[col_detail_url.index(url)]), photo_type='D', image_order=order, photo='items/detail_'+str(col_ga_id[col_detail_url.index(url)])+'_'+str(order)+'.jpg')
        except:            
            pass                                     
                                     
        pk += 1
        
    print(url)    
        

https://www.baeminchan.com/meat/detail.php?hash=HFAE4&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=IE3E6&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=I155E&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=I3E2A&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=ID6FE&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=IF045&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=I4A79&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=IDDA8&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=IEFE4&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=IF329&cno=46100700?cno=46100700
https://www.baeminchan.com/meat/detail.php?hash=ICD4E&cno=46100700?cno=46100700


In [41]:
for pk in range(14474, 14595):
    ItemImage.objects.filter(pk=pk).delete()

In [43]:
for col in col_itemimage:
    print(col)

https://cdn.bmf.kr/_data/product/201702/06/f8a55c7b912935447056cbfed5274c52.jpg
https://cdn.bmf.kr/_data/files/product/14540/948dd15aa40288dff4898d58b517b5af.jpg
https://cdn.bmf.kr/_data/files/product/14540/171612db7a74455d1b05e40ebc7140fc.jpg
https://cdn.bmf.kr/_data/files/product/14540/b245d45fea6fd0a19616e207540d17fc.jpg
https://cdn.bmf.kr/_data/files/product/14540/898ec3deb72946992330d6afbbb540cf.jpg
https://cdn.bmf.kr/_data/files/product/14540/9af40e0e738bce7ee518b6cb48083360.jpg
https://cdn.bmf.kr/_data/files/product/14540/201d2a17e84f598fc2f1bb126002dcaf.jpg
https://cdn.bmf.kr/_data/files/product/14540/63d880b575295e280bcb2ef0389ecc87.jpg
https://cdn.bmf.kr/_data/files/product/14540/f25da672f2d8e868c1fa0f9a652cac0c.jpg
https://cdn.bmf.kr/_data/product/IE3E6/f667ceaa00d0331fd86d308f3644d6fa.jpg
https://cdn.bmf.kr/_data/product/IE3E6/8b58a51ff932caf6cf306773dcafd9ef.jpg
https://cdn.bmf.kr/_data/product/IE3E6/bade35fe1d86a2fd83294d1cb603809a.jpg
https://cdn.bmf.kr/_data/product/IE3

In [ ]:
import urllib.request
image_list = ItemImage.objects.all()
cnt = 0
for col in image_list:
    urllib.request.urlretrieve(col_itemimage[cnt], '../'+col.photo.name)
    cnt += 1
    print(col.photo.name)

In [47]:
ItemImage.objects.filter(Q(pk > 14475))

TypeError: 'bool' object is not iterable

In [50]:
cnt = 0
for pk in range(14474, 14602):
    col_itemimage[cnt]
    urllib.request.urlretrieve(col_itemimage[cnt], '../'+ItemImage.objects.get(pk=pk).photo.name)    
    print(ItemImage.objects.get(pk=14474).photo.name)    
    cnt += 1

items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_thumnail_14540_0.jpg
items/detail_t

In [49]:
ItemImage.objects.get(pk=14474).photo.name

'items/detail_thumnail_14540_0.jpg'

In [62]:
item_list = Item.objects.all()

In [86]:
description_list = []
for col in item_list:
    if not Description.objects.filter(item=col.id).exists():
        description_list.append(col.id)
        print(col.id)

In [88]:
len(description_list)

0

In [23]:
itemimage_list = []
for col in item_list:
    if not ItemImage.objects.filter(item=col.id).exists():
        itemimage_list.append(col.id)
        print(col.id)
    else:
        print('not null')

not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
n

not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
not null
n

In [22]:
ItemImage.objects.filter(item=1028).exists()

True

In [29]:
itemname_list = []
for col in description_list:
    itemname_list.append(Item.objects.get(pk=col).item_name)
    print(Item.objects.get(pk=col).item_name)

나물 3종 세트 (2~3인분) 300g
맛보기 반찬 4종 세트 (2~3인분)
랜덤반찬세트(3~4인분)
고사리나물 250g
파래무침 170g
오징어젓갈 150g
낙지젓갈 150g
동그랑땡 230g
맛보기 세트 (4인분)
건나물 남도 3종세트 (3인분)
타르타르소스 연근샐러드 150g
양념간장 240g
모둠전 5종세트 (5~6인분)  850g
잔멸치볶음 100g
오징어실채 80g
새우계란찜 300g
캐슈넛멸치볶음 60g
새우야채전 300g
떡갈비맛스테이크 (2인분) 280g
스테이크 패밀리팩 4~5인
화이타 코스팩 3~4인
스테이크 코스팩 3~4인
스테이크 파티팩 2인
능이버섯 불고기 (2인분) 1970g
깐새우장 세트 (2종)
떡갈비 (4인분) 320g
갈비찜 550g
김밥 속재료 8종 (5줄분량) 420g
LA갈비 550g (반조리)
소고기버섯샐러드 250g
홍어삼합 (3~4인분) 700g
잣소스닭고기해물냉채 170g
간장 떡볶이 (2~3인분) 810g
문어삼합 (3~4인분) 650g
꽃게범벅 170g
수제가스 버라이어티 세트 (8인분)
교자세트 (3종)
돼지고기 짜글이 찌개	(1~2인분)
구수한 추어탕 480g
소고기미역국(2인분) 800g
소고기순두부찌개 630g
사골곰탕 630g
양지곰탕 630g
소고기배추된장국 630g
소고기뭇국 630g
성게미역국 630g
우렁강된장 270g
소고기육개장국밥 (2인분)
해물짬뽕국밥 (2인분)
작은용량 4종 세트 (7~8인분)
참치두부볼과 허니머스타드소스
배추겉절이 500g
깻잎김치 200g
알타리 1kg
마늘장아찌 200g
마파두부 250g
유니짜장 550g
토마토소스미트볼 350g
함박스테이크 540g
일본카레 3종세트 (3인분)
오리지널 콥샐러드 (1인분) 200g
코울슬로우 200g
프레시바질 콥샐러드 (1인분) 202g
튜나 콥샐러드 (1인분) 200g
그릭요거트 무설탕 (5인분) 500g
그릭요거트 플레인 (5인분) 500g
그릭요거트 무설탕 (3~4인분) 350g
콜드브루와 유리잔
콜드브루 (16~18회분)

In [30]:
len(itemname_list)

79

In [83]:
imagedeatil_list = [
    # 나물 3종 세트 (2~3인분) 300g
    'https://www.baeminchan.com/sidedish/detail.php?cno=46010200&hash=ICF2A',
    # 맛보기 반찬 4종 세트 (2~3인분)
    'https://www.baeminchan.com/sidedish/detail.php?cno=46010700&hash=H50E2',
    # 랜덤반찬세트(3~4인분)
    'https://www.baeminchan.com/sidedish/detail.php?cno=46010700&hash=G61D9',
    # 고사리나물 250g
    'https://www.baeminchan.com/sidedish/detail.php?cno=46010200&hash=H61C4',
    # 파래무침 170g
    'https://www.baeminchan.com/brand/detail.php?cno=30080208&hash=HD59E',
    # 오징어젓갈 150g
    'https://www.baeminchan.com/sidedish/detail.php?cno=46010600&hash=HF596',
    # 낙지젓갈 150g
    'https://www.baeminchan.com/sidedish/detail.php?cno=46010600&hash=H3147',
    # 동그랑땡 230g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030500&hash=G6DBF',
    # 맛보기 세트 (4인분)
    'https://www.baeminchan.com/sidedish/detail.php?cno=46010700&hash=IE8F7',
    # 건나물 남도 3종세트 (3인분)
    'https://www.baeminchan.com/sidedish/detail.php?cno=46010200&hash=IFD7C',
    # 타르타르소스 연근샐러드 150g
    'https://www.baeminchan.com/sidedish/detail.php?cno=46010100&hash=G30FE',
    # 양념간장 240g
    'https://www.baeminchan.com/sidedish/detail.php?cno=46010600&hash=GE5E3',
    # 모둠전 5종세트 (5~6인분)  850g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030500&hash=I0133',
    # 잔멸치볶음 100g
    'https://www.baeminchan.com/brand/detail.php?cno=30080208&hash=HD162',
    # 오징어실채 80g
    'https://www.baeminchan.com/brand/detail.php?cno=30080208&hash=H3738',
    # 새우계란찜 300g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030500&hash=H157C',
    # 캐슈넛멸치볶음 60g
    'https://www.baeminchan.com/sidedish/detail.php?cno=46010300&hash=GE5B3',
    # 새우야채전 300g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030500&hash=GE5F9',
    # 떡갈비맛스테이크 (2인분) 280g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030100&hash=H2A14',
    # 스테이크 패밀리팩 4~5인
    'https://www.baeminchan.com/maindish/detail.php?cno=46030600&hash=I47B3',
    # 화이타 코스팩 3~4인
    'https://www.baeminchan.com/maindish/detail.php?cno=46030600&hash=I2907',
    # 스테이크 코스팩 3~4인
    'https://www.baeminchan.com/maindish/detail.php?cno=46030600&hash=I547A',
    # 스테이크 파티팩 2인
    'https://www.baeminchan.com/maindish/detail.php?cno=46030600&hash=I0CDA',
    # 능이버섯 불고기 (2인분) 1970g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030100&hash=IA22C',
    # 깐새우장 세트 (2종)
    'https://www.baeminchan.com/maindish/detail.php?cno=46030200&hash=I8E6C',
    # 떡갈비 (4인분) 320g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030100&hash=I4BED',
    # 갈비찜 550g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030100&hash=G7075',
    # 김밥 속재료 8종 (5줄분량) 420g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030300&hash=H7015',
    # LA갈비 550g (반조리)
    'https://www.baeminchan.com/maindish/detail.php?cno=46030100&hash=GB859',
    # 소고기버섯샐러드 250g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030100&hash=G1A7F',
    # 홍어삼합 (3~4인분) 700g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030200&hash=I94C7',
    # 잣소스닭고기해물냉채 170g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030100&hash=G4616',
    # 간장 떡볶이 (2~3인분) 810g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030300&hash=I6544',
    # 문어삼합 (3~4인분) 650g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030200&hash=I58B2',
    # 꽃게범벅 170g
    'https://www.baeminchan.com/maindish/detail.php?cno=46030200&hash=H3F46',    
    # 교자세트 (3종)
    'https://www.baeminchan.com/maindish/detail.php?cno=46030300&hash=HC374',
    # 돼지고기 짜글이 찌개	(1~2인분)
    'https://www.baeminchan.com/soups/detail.php?cno=46060200&hash=I95F9',
    # 구수한 추어탕 480g
    'https://www.baeminchan.com/soups/detail.php?cno=46060300&hash=I02A6',
    # 소고기미역국(2인분) 800g
    'https://www.baeminchan.com/soups/detail.php?cno=46060100&hash=H0B5D',
    # 소고기순두부찌개 630g
    'https://www.baeminchan.com/soups/detail.php?cno=46060200&hash=H6349',
    # 사골곰탕 630g
    'https://www.baeminchan.com/soups/detail.php?cno=46060300&hash=HCF78',
    # 양지곰탕 630g
    'https://www.baeminchan.com/soups/detail.php?cno=46060300&hash=GA0B2',
    # 소고기배추된장국 630g
    'https://www.baeminchan.com/soups/detail.php?cno=46060100&hash=G2F45',
    # 소고기뭇국 630g
    'https://www.baeminchan.com/soups/detail.php?cno=46060100&hash=GE639',
    # 성게미역국 630g
    'https://www.baeminchan.com/soups/detail.php?cno=46060100&hash=G6F3C',
    # 우렁강된장 270g
    'https://www.baeminchan.com/soups/detail.php?cno=46060200&hash=G12E5',
    # 소고기육개장국밥 (2인분)
    'https://www.baeminchan.com/soups/detail.php?cno=46060100&hash=I87F8',
    # 해물짬뽕국밥 (2인분)
    'https://www.baeminchan.com/soups/detail.php?cno=46060100&hash=IE7A6',
    # 작은용량 4종 세트 (7~8인분)
    'https://www.baeminchan.com/soups/detail.php?cno=46060500&hash=ID966',
    # 참치두부볼과 허니머스타드소스
    'https://www.baeminchan.com/forkids/detail.php?cno=46070200&hash=I10DC',
    # 배추겉절이 500g
    'https://www.baeminchan.com/pickles/detail.php?cno=46020100&hash=H14DB',
    # 깻잎김치 200g
    'https://www.baeminchan.com/pickles/detail.php?cno=46020100&hash=H1366',
    # 알타리 1kg
    'https://www.baeminchan.com/pickles/detail.php?cno=46020100&hash=HD352',
    # 마늘장아찌 200g
    'https://www.baeminchan.com/pickles/detail.php?cno=46020200&hash=G6262',
    # 마파두부 250g
    'https://www.baeminchan.com/foreign/detail.php?cno=46040200&hash=G5462',
    # 유니짜장 550g
    'https://www.baeminchan.com/foreign/detail.php?cno=46040200&hash=GB536',
    # 토마토소스미트볼 350g
    'https://www.baeminchan.com/foreign/detail.php?cno=46040100&hash=G1AC4',
    # 함박스테이크 540g
    'https://www.baeminchan.com/foreign/detail.php?cno=46040100&hash=H314D',
    # 일본카레 3종세트 (3인분)
    'https://www.baeminchan.com/rice-noodle/detail.php?cno=46050100&hash=I33EE',
    # 오리지널 콥샐러드 (1인분) 200g
    'https://www.baeminchan.com/salad/detail.php?cno=46080100&hash=GA616',
    # 코울슬로우 200g
    'https://www.baeminchan.com/salad/detail.php?cno=46080100&hash=GFC36',
    # 프레시바질 콥샐러드 (1인분) 202g
    'https://www.baeminchan.com/salad/detail.php?cno=46080100&hash=G0AB7',
    # 튜나 콥샐러드 (1인분) 200g
    'https://www.baeminchan.com/salad/detail.php?cno=46080100&hash=G5138',
    # 그릭요거트 무설탕 (5인분) 500g
    'https://www.baeminchan.com/snack/detail.php?cno=46120500&hash=I2F7F',
    # 그릭요거트 플레인 (5인분) 500g
    'https://www.baeminchan.com/snack/detail.php?cno=46120500&hash=I093B',
    # 그릭요거트 무설탕 (3~4인분) 350g
    'https://www.baeminchan.com/snack/detail.php?cno=46120500&hash=IDD92',   
    # 콤비네이션피자 415g
    'https://www.baeminchan.com/snack/detail.php?cno=46120600&hash=I9F2E',
    # 찰핫도그 80g
    'https://www.baeminchan.com/snack/detail.php?cno=46120600&hash=IC927',
    # 너겟 550g
    'https://www.baeminchan.com/snack/detail.php?cno=46120600&hash=H48AD',
    # 그릭요거트 플레인 (3~4인분) 350g
    'https://www.baeminchan.com/snack/detail.php?cno=46120500&hash=I4AC7',    
]

In [41]:
Item.objects.get(item_name='수제가스 버라이어티 세트 (8인분)').id

427

In [42]:
Item.objects.get(item_name='콜드브루와 유리잔').id

1070

In [43]:
Item.objects.get(item_name='콜드브루 (16~18회분) 1L').id

1071

In [44]:
Item.objects.get(item_name='티백 (5gX10팩)').id

1072

In [45]:
Item.objects.get(item_name='드립백 (11gX6팩)').id

1073

In [46]:
Item.objects.get(item_name='콜드브루 화이트 (2~4회분) 180ml').id

1074

In [47]:
Item.objects.get(item_name='틴케이스 선물세트').id

1075

In [48]:
Item.objects.get(item_name='콜드브루 골드 (2~4회분) 180ml').id

1076

In [50]:
Item.objects.get(item_name='홈커피 선물세트').id

1077

In [61]:
pk_list  = [427,1070,1071,1072,1073,1074,1075,1076,1077]
for col in pk_list:
    Item.objects.filter(pk=col).delete()
    print(col)

427
1070
1071
1072
1073
1074
1075
1076
1077


In [65]:
ga_id_list = []
for col in description_list:
    ga_id_list.append(Item.objects.get(pk=col).ga_id)
    print(Item.objects.get(pk=col).ga_id)

17703
17476
12430
15107
15104
15041
15042
11972
18371
17889
12622
13142
18457
15030
15033
17192
11966
11965
14366
18746
18745
18744
18743
18452
17987
18482
12476
16760
12477
12728
18368
12620
18089
18369
14877
17177
18064
17731
16550
14878
14871
13136
12727
12136
12134
11967
18040
18038
17931
17997
15100
15094
15096
12723
11956
13140
12623
17172
18616
7977
13143
11010
7979
18560
18562
18559
18059
18377
17176
18561


In [85]:
from bs4 import BeautifulSoup
import requests, re

col_detail_url = imagedeatil_list
col_ga_id = ga_id_list

for url in col_detail_url :    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    soup_detail = soup.find('div', id='app')

    added_words = ''
    try:
        added_words = soup_detail.find_all("p", "desc_bt_txt")[0].get_text()
    except:
        pass

    point = 0
    try:
        int(soup_detail.find_all("dl", "desc_info")[0].select('dd')[0].get_text().replace('원',''))
    except:
        pass
    else:
        point = int(soup_detail.find_all("dl", "desc_info")[0].select('dd')[0].get_text().replace('원',''))

    delivery_type = ''
    try:        
        delivery_type = soup_detail.find_all("dl", "desc_info")[0].select('dd')[1].get_text().replace('\r','').replace('\n','')
    except:
        pass                    

    receive_day = ''
    try:
        receive_day = soup_detail.find_all("dl", "desc_info")[0].select('dd')[2].get_text().replace('\r','').replace('\n','')
    except:
        pass

    regular_delivery = ''
    try:
        regular_delivery = soup_detail.find_all("dl", "desc_info")[0].select('dd')[3].get_text()
    except:
        pass

    if regular_delivery == '가능':
        regular_delivery = True
    else:
        regular_delivery = False

    soup_detail_description = soup.find('div', id='detail_official_information')
    col_is_equal = [
        '식품의 유형',
        '생산자 및 소재지(수입품의 경우 수입자를 함께 표기)',
        '제조연월일, 유통기한 또는 품질유지기한',
        '포장단위별 용량(중량), 수량',
        '원재료명 및 함량',
        '영양성분',
        '유전자재조합식품 여부',
        '영유아식 또는 체중조절식품 등에 해당하는 경우',
        '수입식품 문구',
        '소비자상담 관련 전화번호',    
        '알레르기 주의사항',
        '알레르기 유발물질',
        '품목신고번호',
    ]
    col_th = []
    col_td = []

    col_th = soup_detail_description.find_all('tbody')[0].select('th')
    col_td = soup_detail_description.find_all('tbody')[0].select('td')

    col_description_data = []

    for col in col_is_equal:
        descripiton = ''
        for col_th_num in col_th:
            if col == col_th_num.get_text():
                descripiton = col_td[col_th.index(col_th_num)].get_text().replace('\r','').replace('\n','')
                break

        col_description_data.append(descripiton)
        
    print(col_detail_url.index(url),col_ga_id[col_detail_url.index(url)])  
    Description.objects.create(
        item=Item.objects.get(ga_id=col_ga_id[col_detail_url.index(url)]), 
        added_words=added_words,
        point=point,
        delivery_type=delivery_type,
        receive_day=receive_day,
        regular_delivery=regular_delivery,
        item_type=col_description_data[0],
        factory_address=col_description_data[1],
        dom=col_description_data[2],
        capacity=col_description_data[3],
        ingredient=col_description_data[4],
        caution=col_description_data[10],                
        allergy_material=col_description_data[11],        
    )
        

0 17703
1 17476
2 12430
3 15107
4 15104
5 15041
6 15042
7 11972
8 18371
9 17889
10 12622
11 13142
12 18457
13 15030
14 15033
15 17192
16 11966
17 11965
18 14366
19 18746
20 18745
21 18744
22 18743
23 18452
24 17987
25 18482
26 12476
27 16760
28 12477
29 12728
30 18368
31 12620
32 18089
33 18369
34 14877
35 17177
36 18064
37 17731
38 16550
39 14878
40 14871
41 13136
42 12727
43 12136
44 12134
45 11967
46 18040
47 18038
48 17931
49 17997
50 15100
51 15094
52 15096
53 12723
54 11956
55 13140
56 12623
57 17172
58 18616
59 7977
60 13143
61 11010
62 7979
63 18560
64 18562
65 18559
66 18059
67 18377
68 17176
69 18561


In [69]:
Item.objects.get(pk=695).item_name

'돼지 반마리 오마카세 (2~3인분) 400g'

In [84]:
for col in description_list:
    Description.objects.filter(item=col).delete()
    print(col)

46
72
103
114
115
127
149
161
167
171
174
185
194
197
212
228
229
238
242
243
244
245
246
256
285
327
330
352
361
362
375
377
379
385
388
442
474
485
523
548
549
553
554
555
556
557
582
583
586
627
796
801
816
823
883
895
901
920
937
985
986
991
993
1010
1016
1020
1078
1079
1080
1028


In [2]:
item_list = [
    46,
    72,
    103,
    114,
    115,
    127,
    149,
    161,
    167,
    171,
    174,
    185,
    194,
    197,
    212,
    228,
    229,
    238,
    242,
    243,
    244,
    245,
    246,
    256,
    285,
    327,
    330,
    352,
    361,
    362,
    375,
    377,
    379,
    385,
    388,
    442,
    474,
    485,
    523,
    548,
    549,
    553,
    554,
    555,
    556,
    557,
    582,
    583,
    586,
    627,
    796,
    801,
    816,
    823,
    883,
    895,
    901,
    920,
    937,
    985,
    986,
    991,
    993,
    1010,
    1016,
    1020,
    1078,
    1079,
    1080,
    1028,
]

In [4]:
for col in item_list:
    print('https://baeminchan.netlify.com/item/?item_pk='+str(col))

https://baeminchan.netlify.com/item/?item_pk=46
https://baeminchan.netlify.com/item/?item_pk=72
https://baeminchan.netlify.com/item/?item_pk=103
https://baeminchan.netlify.com/item/?item_pk=114
https://baeminchan.netlify.com/item/?item_pk=115
https://baeminchan.netlify.com/item/?item_pk=127
https://baeminchan.netlify.com/item/?item_pk=149
https://baeminchan.netlify.com/item/?item_pk=161
https://baeminchan.netlify.com/item/?item_pk=167
https://baeminchan.netlify.com/item/?item_pk=171
https://baeminchan.netlify.com/item/?item_pk=174
https://baeminchan.netlify.com/item/?item_pk=185
https://baeminchan.netlify.com/item/?item_pk=194
https://baeminchan.netlify.com/item/?item_pk=197
https://baeminchan.netlify.com/item/?item_pk=212
https://baeminchan.netlify.com/item/?item_pk=228
https://baeminchan.netlify.com/item/?item_pk=229
https://baeminchan.netlify.com/item/?item_pk=238
https://baeminchan.netlify.com/item/?item_pk=242
https://baeminchan.netlify.com/item/?item_pk=243
https://baeminchan.net